# 2. Sistemas de Memoria Avanzados en LangChain

## Objetivos de Aprendizaje
- Comprender las limitaciones de la gestión manual de la memoria.
- Aprender a usar las clases de memoria de LangChain para automatizar el historial de chat.
- Implementar `ConversationBufferMemory` para un historial completo.
- Implementar `ConversationBufferWindowMemory` para limitar el tamaño del historial.
- Implementar `ConversationSummaryMemory` para resumir conversaciones y ahorrar tokens.
- Entender el puente hacia la planificación de agentes.

## El Problema con la Memoria Manual

En el notebook anterior, le dimos memoria a nuestro agente gestionando manualmente una lista `chat_history`. Aunque funcional, este enfoque tiene dos grandes inconvenientes:

1.  **Gestión Tediosa**: Actualizar la lista manualmente después de cada interacción es propenso a errores y no es escalable.
2.  **Límite de Contexto**: Enviar el historial completo en cada llamada consume tokens rápidamente. En conversaciones largas, esto puede exceder el límite de contexto del modelo, provocando errores o un alto costo.

LangChain resuelve esto con un sistema de **clases de Memoria** que automatizan el proceso y ofrecen estrategias para gestionar historiales largos.

### 1. Instalación y Configuración

In [ ]:
!pip install langchain langchain-openai openai wikipedia -q

In [1]:
import os
import wikipedia
from langchain_openai import ChatOpenAI

# Configurar el idioma de Wikipedia
wikipedia.set_lang("es")

# Configuración del LLM
try:
    llm = ChatOpenAI(
        model="gpt-4o",
        openai_api_base=os.environ.get("GITHUB_BASE_URL"),
        openai_api_key=os.environ.get("GITHUB_TOKEN"),
        temperature=0
    )
    print("✅ LLM de LangChain configurado.")
except Exception as e:
    print(f"❌ Error configurando el LLM: {e}")
    llm = None

✅ LLM de LangChain configurado.


### 2. Agente y Herramientas (Sin Cambios)

Reutilizamos el mismo agente y herramientas. La innovación estará en cómo gestionamos la memoria.

In [2]:
from langchain.agents import tool, create_openai_tools_agent, AgentExecutor
from langchain import hub

@tool
def get_wikipedia_summary(query: str) -> str:
    """Busca en Wikipedia un tema y devuelve un resumen de 2 frases. Útil para obtener información sobre personas, lugares o conceptos."""
    try:
        return wikipedia.summary(query, sentences=2)
    except Exception as e:
        return f"Ocurrió un error: {e}"

tools = [get_wikipedia_summary]

prompt = hub.pull("hwchase17/openai-tools-agent")

agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

print("✅ Agente y herramientas listos.")

✅ Agente y herramientas listos.


### 3. Automatizando con `ConversationBufferMemory`

Esta es la memoria más básica. Simplemente almacena todos los mensajes en una variable (el "buffer") y los pasa en cada llamada. Replica lo que hicimos manualmente, pero de forma automática.

In [3]:
from langchain.memory import ConversationBufferMemory

# memory_key debe coincidir con la variable de entrada en el prompt ('chat_history')
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

def chat_with_agent_buffer(query: str):
    # Carga el historial de la memoria
    history = memory.load_memory_variables({})["chat_history"]
    
    # Invoca al agente con el historial
    response = agent_executor.invoke({"input": query, "chat_history": history})
    
    # Guarda el nuevo par de pregunta/respuesta en la memoria
    memory.save_context({"input": query}, {"output": response["output"]})
    
    return response["output"]

C:\Users\Alejandro\AppData\Local\Temp\ipykernel_10024\442967770.py:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


In [4]:
print("Primera pregunta...")
response1 = chat_with_agent_buffer("Háblame del planeta Marte")
print(f"Respuesta 1: {response1}")

print("Segunda pregunta de seguimiento...")
response2 = chat_with_agent_buffer("¿Tiene alguna luna?")
print(f"Respuesta 2: {response2}")

Primera pregunta...


> Entering new AgentExecutor chain...

Invoking: `get_wikipedia_summary` with `{'query': 'Marte'}`




c:\Users\Alejandro\Documents\GitHub\productos_app\Ingenier-a-de-Soluciones-con-Inteligencia-Artificial\.venv\Lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file c:\Users\Alejandro\Documents\GitHub\productos_app\Ingenier-a-de-Soluciones-con-Inteligencia-Artificial\.venv\Lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


Ocurrió un error: "Marte" may refer to: 
Marte (planeta)
Marte (mitología)
hierro
Marte (Velázquez)
Marte, el portador de la guerra
The Martian
Marte (serie de televisión)
«Marte» (canción)
Marte (Coahuila)
Marte (Nigeria)
Club Deportivo Atlético Marte
Club Deportivo Marte
Wikcionario
Diccionario de la Real Academia Española
Invoking: `get_wikipedia_summary` with `{'query': 'Marte planeta'}`


Marte es el cuarto planeta en orden de distancia al Sol y el segundo más pequeño del sistema solar, después de Mercurio. Recibió su nombre en homenaje al homónimo dios de la guerra de la mitología romana (Ares en la mitología griega), y también es conocido como «el planeta rojo»​​ debido a la apariencia rojiza​ que le confiere el óxido de hierro predominante en su superficie.Marte es el cuarto planeta del sistema solar y el segundo más pequeño, después de Mercurio. Conocido como "el planeta rojo" por el óxido de hierro en su superficie, su nombre proviene del dios romano de la guerra, Marte (Ares

Como puedes ver, la función `chat_with_agent_buffer` ahora se encarga de cargar y guardar el historial, haciendo el proceso mucho más limpio.

### 4. Gestionando el Contexto con `ConversationBufferWindowMemory`

Para evitar que el historial crezca sin control, podemos usar `ConversationBufferWindowMemory`. Esta memoria solo conserva un número `k` de las últimas interacciones.

Esto es un buen equilibrio entre tener contexto y no sobrecargar al LLM.

In [5]:
from langchain.memory import ConversationBufferWindowMemory

# Creamos una nueva memoria, esta vez con k=1 (solo recuerda el último par de mensajes)
memory_window = ConversationBufferWindowMemory(k=1, memory_key="chat_history", return_messages=True)

def chat_with_agent_window(query: str):
    history = memory_window.load_memory_variables({})["chat_history"]
    response = agent_executor.invoke({"input": query, "chat_history": history})
    memory_window.save_context({"input": query}, {"output": response["output"]})
    return response["output"]

C:\Users\Alejandro\AppData\Local\Temp\ipykernel_10024\1328021610.py:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory_window = ConversationBufferWindowMemory(k=1, memory_key="chat_history", return_messages=True)


In [6]:
print("Pregunta 1: ¿Quién fue Albert Einstein?")
chat_with_agent_window("¿Quién fue Albert Einstein?")

print("Pregunta 2: ¿Y Niels Bohr?")
chat_with_agent_window("¿Y Niels Bohr?")

print("Pregunta 3: ¿De qué trataba la primera pregunta que te hice?")
response3 = chat_with_agent_window("¿De qué trataba la primera pregunta que te hice?")
print(f"Respuesta 3: {response3}")

Pregunta 1: ¿Quién fue Albert Einstein?


> Entering new AgentExecutor chain...

Invoking: `get_wikipedia_summary` with `{'query': 'Albert Einstein'}`


Albert Einstein pronunciación en alemán: /ˈalbɐt ˈaɪnʃtaɪn/ ();​ (Ulm, Imperio alemán, 14 de marzo de 1879-Princeton, Estados Unidos, 18 de abril de 1955) fue un físico alemán de origen judío, nacionalizado después suizo, austriaco y estadounidense. Se le considera el científico más importante, conocido y popular del siglo XX.​​
En 1905, cuando era un joven físico desconocido, empleado en la Oficina de Patentes de Berna, publicó su teoría de la relatividad especial.Albert Einstein fue un físico alemán de origen judío, nacionalizado suizo, austriaco y estadounidense, considerado el científico más importante y popular del siglo XX. En 1905, mientras trabajaba en la Oficina de Patentes de Berna, publicó su teoría de la relatividad especial, revolucionando la física.

> Finished chain.
Pregunta 2: ¿Y Niels Bohr?


> Entering new AgentExecu

El agente probablemente no podrá responder a la tercera pregunta, porque con `k=1`, la interacción sobre Einstein ya fue descartada de la memoria para dar paso a la de Niels Bohr.

### 5. Ahorrando Tokens con `ConversationSummaryMemory`

Esta es la estrategia más avanzada. En lugar de guardar los mensajes completos, utiliza un LLM para crear un resumen de la conversación a medida que avanza. En cada nueva interacción, el resumen se actualiza y se pasa como contexto.

Es ideal para conversaciones muy largas donde los detalles específicos se vuelven menos importantes que el contexto general.

In [7]:
from langchain.memory import ConversationSummaryMemory

# Esta memoria necesita un LLM para hacer los resúmenes
memory_summary = ConversationSummaryMemory(llm=llm, memory_key="chat_history", return_messages=True)

def chat_with_agent_summary(query: str):
    history = memory_summary.load_memory_variables({})["chat_history"]
    response = agent_executor.invoke({"input": query, "chat_history": history})
    memory_summary.save_context({"input": query}, {"output": response["output"]})
    return response["output"]

C:\Users\Alejandro\AppData\Local\Temp\ipykernel_10024\1927192588.py:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory_summary = ConversationSummaryMemory(llm=llm, memory_key="chat_history", return_messages=True)


In [8]:
print("Pregunta 1: Necesito organizar un viaje a Japón. ¿Cuál es la mejor época para ir?")
chat_with_agent_summary("Necesito organizar un viaje a Japón. ¿Cuál es la mejor época para ir?")

print("Pregunta 2: Suena bien. ¿Qué ciudades me recomiendas visitar en un primer viaje?")
chat_with_agent_summary("¿Qué ciudades me recomiendas visitar en un primer viaje?")

print("Pregunta 3: ¿Sobre qué estábamos hablando?")
response3_summary = chat_with_agent_summary("¿Sobre qué estábamos hablando?")
print(f"Respuesta 3: {response3_summary}")

Pregunta 1: Necesito organizar un viaje a Japón. ¿Cuál es la mejor época para ir?


> Entering new AgentExecutor chain...

Invoking: `get_wikipedia_summary` with `{'query': 'mejor época para viajar a Japón'}`


Ocurrió un error: Page id "mejor época para villar a japón" does not match any pages. Try another id!La mejor época para viajar a Japón depende de tus intereses, pero generalmente se recomienda:

1. **Primavera (marzo a mayo):** Es famosa por los cerezos en flor (sakura), especialmente a finales de marzo y principios de abril. El clima es templado y agradable.

2. **Otoño (septiembre a noviembre):** Es ideal para disfrutar del follaje otoñal, con colores vibrantes en los árboles. El clima también es fresco y cómodo.

Ambas estaciones ofrecen paisajes espectaculares y temperaturas agradables. Evita el verano (junio a agosto) si no te gusta el calor y la humedad, y ten en cuenta que el invierno puede ser frío, aunque tiene su encanto en las regiones montañosas.

> Finished chain.


El agente debería poder responder correctamente, ya que la memoria no contiene los mensajes literales, sino un resumen de que el usuario está planeando un viaje a Japón.

## Conclusiones y Transición al Módulo IL2.3

Hemos visto cómo las clases de memoria de LangChain nos permiten crear agentes conversacionales mucho más robustos y eficientes, superando los desafíos de la gestión manual y el tamaño del contexto.

**Resumen de Estrategias de Memoria:**
- **`ConversationBufferMemory`**: Simple y completa. Ideal para conversaciones cortas donde cada detalle importa.
- **`ConversationBufferWindowMemory`**: Eficiente y balanceada. Perfecta para chatbots de servicio al cliente que necesitan contexto reciente.
- **`ConversationSummaryMemory`**: Ahorradora de tokens y escalable. La mejor opción para asistentes de largo plazo y análisis de conversaciones extensas.

Con un agente que puede recordar de manera efectiva, hemos completado una pieza fundamental del rompecabezas. Sin embargo, las conversaciones del mundo real a menudo implican tareas que no se resuelven con una sola herramienta o en un solo paso.

**¿Qué pasa si el agente necesita buscar información, luego calcular algo con esa información y finalmente enviar un correo electrónico?**

Esto requiere que el agente pueda **planificar** una secuencia de acciones. Debe ser capaz de descomponer un objetivo complejo en pasos más pequeños y ejecutarlos en el orden correcto.

Esta es la transición perfecta al **Módulo IL2.3: Planificación y Orquestación de Agentes**, donde exploraremos cómo los agentes pueden crear, seguir y adaptar planes para resolver problemas complejos y multi-paso.